In [1]:
import tensorflow as tf

In [2]:
tf.__version__

'1.2.1'

In [3]:
node1 = tf.constant(3.0,dtype=tf.float32)
node2 = tf.constant(4.0) #also tf.float32 implicitly
print(node1,node2)

Tensor("Const:0", shape=(), dtype=float32) Tensor("Const_1:0", shape=(), dtype=float32)


In [4]:
session = tf.Session()
print(session.run([node1,node2]))

[3.0, 4.0]


In [5]:
node3=tf.add(node1,node2)
print("Node3:",node3)
print("session.run(node3)",session.run(node3))

Node3: Tensor("Add:0", shape=(), dtype=float32)
session.run(node3) 7.0


In [6]:
a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)
adder_node = a+b # + provides a shortcut for tf.add(a,b)

In [9]:
print(session.run(adder_node,{a:5,b:4.5}))
print(session.run(adder_node,{a:[2,5],b:[2,4]}))

9.5
[ 4.  9.]


In [10]:
add_and_triple = adder_node*3
print(session.run(add_and_triple,{a:3,b:4.5}))

22.5


In [28]:
m = tf.Variable([.3],dtype=tf.float32)
c = tf.Variable([-.3],dtype=tf.float32)
x = tf.placeholder(tf.float32)
linear_model = m*x + c 

In [29]:
init = tf.global_variables_initializer()
session.run(init)

In [30]:
print(session.run(linear_model,{x:[1,2,3,4]}))

[ 0.          0.30000001  0.60000002  0.90000004]


In [31]:
y = tf.placeholder(tf.float32)
squared_deltas = tf.square(linear_model-y)
loss = tf.reduce_sum(squared_deltas)
print(session.run(loss,{x:[1,2,3,4],y:[0,-1,-2,-3]}))

23.66


In [34]:
fixm = tf.assign(m,[-1.])
fixc = tf.assign(c,[1.])
session.run([fixm,fixc])
print(session.run(loss,{x:[1,2,3,4],y:[0,-1,-2,-3]}))

0.0


In [ ]:
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)
session.run(init)
for i in range(1000):
    session.run(train,{x:[1,2,3,4],y:[0,-1,-2,-3]})

print(session.run([m,c]))

# A Trainable linear regression model

In [3]:
import tensorflow as tf

# Model parameters
W = tf.Variable([.3], dtype=tf.float32)
b = tf.Variable([-.3], dtype=tf.float32)
# Model input and output
x = tf.placeholder(tf.float32)
linear_model = W * x + b
y = tf.placeholder(tf.float32)

# loss
loss = tf.reduce_sum(tf.square(linear_model - y)) # sum of the squares
# optimizer
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

# training data
x_train = [1, 2, 3, 4]
y_train = [0, -1, -2, -3]
# training loop
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init) # reset values to wrong
for i in range(1000):
  sess.run(train, {x: x_train, y: y_train})

# evaluate training accuracy
curr_W, curr_b, curr_loss = sess.run([W, b, loss], {x: x_train, y: y_train})
print("W: %s b: %s loss: %s"%(curr_W, curr_b, curr_loss))

W: [-0.9999969] b: [ 0.99999082] loss: 5.69997e-11


In [2]:
import tensorflow as tf
# NumPy is often used to load, manipulate and preprocess data.
import numpy as np

# Declare list of features. We only have one numeric feature. There are many
# other types of columns that are more complicated and useful.
feature_columns = [tf.feature_column.numeric_column("x", shape=[1])]

# An estimator is the front end to invoke training (fitting) and evaluation
# (inference). There are many predefined types like linear regression,
# linear classification, and many neural network classifiers and regressors.
# The following code provides an estimator that does linear regression.
estimator = tf.estimator.LinearRegressor(feature_columns=feature_columns)

# TensorFlow provides many helper methods to read and set up data sets.
# Here we use two data sets: one for training and one for evaluation
# We have to tell the function how many batches
# of data (num_epochs) we want and how big each batch should be.
x_train = np.array([1., 2., 3., 4.])
y_train = np.array([0., -1., -2., -3.])
x_eval = np.array([2., 5., 8., 1.])
y_eval = np.array([-1.01, -4.1, -7, 0.])
input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_train}, y_train, batch_size=4, num_epochs=None, shuffle=True)
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_train}, y_train, batch_size=4, num_epochs=1000, shuffle=False)
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_eval}, y_eval, batch_size=4, num_epochs=1000, shuffle=False)

# We can invoke 1000 training steps by invoking the  method and passing the
# training data set.
estimator.train(input_fn=input_fn, steps=1000)

# Here we evaluate how well our model did.
train_metrics = estimator.evaluate(input_fn=train_input_fn)
eval_metrics = estimator.evaluate(input_fn=eval_input_fn)
print("train metrics: %r"% train_metrics)
print("eval metrics: %r"% eval_metrics)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpt91wpid3', '_tf_random_seed': 1, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmpt91wpid3/model.ckpt.
INFO:tensorflow:loss = 11.0, step = 1
INFO:tensorflow:global_step/sec: 591.766
INFO:tensorflow:loss = 0.261995, step = 101 (0.170 sec)
INFO:tensorflow:global_step/sec: 562.573
INFO:tensorflow:loss = 0.0353081, step = 201 (0.178 sec)
INFO:tensorflow:global_step/sec: 641.361
INFO:tensorflow:loss = 0.0153921, step = 301 (0.156 sec)
INFO:tensorflow:global_step/sec: 555.904
INFO:tensorflow:loss = 0.00285832, step = 401 (0.180 sec)
INFO:tensorflow:global_step/sec: 649.45
INFO:tensorflow:loss = 0.000153637, step = 501 (0.154 sec)
INFO:tensorf